In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.3f}')
pd.set_option('display.max_colwidth', 100)

import numpy as np 
from matplotlib import pyplot as plt

%pip install -r requirements.txt
from google.cloud import bigquery
pmc = "`lemoncake-prod.serving_layer.modelo_para_calculos_finais_controle_recebimentos_join_cobrancas_dynamo`"
project_id = 'lemoncake-prod'
client = bigquery.Client(project = project_id)

%pip install plotly==5.17.0
import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime, timedelta

ERROR: Ignored the following versions that require a different python version: 2.10.0 Requires-Python >=3.6, <3.10; 2.11.0 Requires-Python >=3.6, <3.10; 2.12.0 Requires-Python >=3.6, <3.10; 2.13.0 Requires-Python >=3.6, <3.10; 2.13.1 Requires-Python >=3.6, <3.10; 2.14.0 Requires-Python >=3.6, <3.10; 2.15.0 Requires-Python >=3.6, <3.10; 2.16.0 Requires-Python >=3.6, <3.10; 2.16.1 Requires-Python >=3.6, <3.10; 2.17.0 Requires-Python >=3.6, <3.10; 2.18.0 Requires-Python >=3.6, <3.10; 2.19.0 Requires-Python >=3.6, <3.10; 2.20.0 Requires-Python >=3.6, <3.10; 2.21.0 Requires-Python >=3.6, <3.10; 2.22.0 Requires-Python >=3.6, <3.10; 2.22.1 Requires-Python >=3.6, <3.10; 2.23.0 Requires-Python >=3.6, <3.10; 2.23.1 Requires-Python >=3.6, <3.10; 2.23.2 Requires-Python >=3.6, <3.10; 2.23.3 Requires-Python >=3.6, <3.10; 2.24.0 Requires-Python >=3.6, <3.10; 2.24.1 Requires-Python >=3.6, <3.10; 2.25.0 Requires-Python >=3.6, <3.10; 2.25.1 Requires-Python >=3.6, <3.10; 2.25.2 Requires-Python >=3.6, <3.

Note: you may need to restart the kernel to use updated packages.


c:\Users\DELL\anaconda3\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
my_query = ''' DECLARE
  currentReferenceMonth STRING;
SET
  currentReferenceMonth = CONCAT(EXTRACT(year
    FROM
      CURRENT_DATE()), '-', EXTRACT(month
    FROM
      CURRENT_DATE()), '-01');
WITH
  referenceMonths AS (
  SELECT
    *
  FROM
    UNNEST(['2023-09-01', '2023-10-01', '2023-11-01', currentReferenceMonth]) AS baseReferenceMonth ),
  subscriber AS (
  SELECT
    disco_consumer_unit_id,
    disco_client_id,
    DATE(CONCAT(EXTRACT(year
        FROM
          subs.create_at), '-', EXTRACT(month
        FROM
          subs.create_at), '-01')) AS subscriptionReferenceMonth,
    invoicing_plan_id,
  FROM
    `lemoncake-prod.full_layer.dynamodb_finance_graph_subscriptions_full` subs
  JOIN
    `lemoncake-prod.full_layer.dynamodb_consortiums_graph_consumer_units_full` consUnit
  ON
    consUnit.id = JSON_VALUE(subscriber, '$.id')
  WHERE
    product = 'singleBill'
    AND status = 'active'),
  subscriberByReferenceMonth AS (
  SELECT
    *
  FROM
    referenceMonths
  JOIN
    subscriber
  ON
    DATE(baseReferenceMonth) >= subscriptionReferenceMonth),
  allData AS (
  SELECT
    CASE
      WHEN (COALESCE(JSON_EXTRACT(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields'), JSON_EXTRACT(discobill.other_info_in_dynamodb, '$.xml.parsed'), ocr_result, file_key)) IS NULL THEN 'Sem dados obtidos'
    ELSE
    'Dados obtidos'
  END
    AS status_download_conta,
    COALESCE(JSON_VALUE(discobill.other_info_in_dynamodb, '$.xml.parsed.barcodeNumber'), JSON_VALUE(ocr_result, '$.parsed.fields.barcodeNumber.value'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields.barcodeNumber.value')) AS barCode,
    COALESCE(JSON_VALUE(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields.pixCode.value')) AS Pix,
    invoice.amount,
    invoice.id,
    invoice.status,
    invoice.invoice_delivered,
    invoice.due_date,
    JSON_VALUE(boleto.integration, '$.externalBoleto.nossoNumero') AS nossoNumero,
    JSON_VALUE(boleto.integration, '$.externalBoleto.sacado.nome') AS razaoSocial,
    subscriberByReferenceMonth.disco_consumer_unit_id,
    subscriberByReferenceMonth.disco_client_id,
    subscriberByReferenceMonth.subscriptionReferenceMonth,
    JSON_VALUE(payment_info, '$.paid.total') AS total_paid,
    JSON_VALUE(other_info, '$.discoBillCost') AS valorDisco,
    JSON_VALUE(other_info, '$.lemonBillCost') AS valorLemon,
    consortiums.id AS consortiumId,
    `lemoncake-prod.serving_layer.getCodConsorcioPorUsinaOuNomeConsorcio`(full_name) AS codigo_consorcio_finance,
    COALESCE(JSON_VALUE(other_info, '$.distributionCompany'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.xml.parsed.billDisco'), JSON_VALUE(ocr_result, '$.parsed.fields.billDisco.value'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields.billDisco.value')) AS disco,
    COALESCE(JSON_VALUE(other_info, '$.discoBillDueDate'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.xml.parsed.discoBillDueDate'), JSON_VALUE(ocr_result, '$.parsed.fields.discoBillDueDate.value'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields.discoBillDueDate.value')) AS discoDueDate,
    COALESCE(JSON_VALUE(invoice.period, '$.reference'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.xml.parsed.billReferenceMonth'), JSON_VALUE(ocr_result, '$.parsed.fields.billReferenceMonth.value'), JSON_VALUE(discobill.other_info_in_dynamodb, '$.apiResponse.parsed.fields.billReferenceMonth.value')) AS refMonth,
    baseReferenceMonth,
    JSON_VALUE(boleto.other_info_in_dynamodb, '$.paymentInfo.paymentDate') paymentDate,
  FROM
    subscriberByReferenceMonth
  LEFT JOIN
    `lemoncake-prod.full_layer.dynamodb_disco_bills_graph_disco_bill_files_full` discobill
  ON
    disco_consumer_unit_id = JSON_VALUE(discobill.other_info_in_dynamodb, '$.primaryIdentifier')
    AND SAFE_CAST(JSON_VALUE(discobill.other_info_in_dynamodb, '$.referenceMonth') AS date) = DATE(baseReferenceMonth)
  LEFT JOIN
    `lemoncake-prod.full_layer.dynamodb_finance_graph_charges_full` charge
  ON
    disco_consumer_unit_id = JSON_VALUE(other_info, '$.discoConsumerUnitId')
    AND charge.product = 'businessDistributedEnergyGeneration'
    AND SAFE_CAST(JSON_VALUE(period, '$.reference') AS date) = SAFE_CAST(JSON_VALUE(discobill.other_info_in_dynamodb, '$.referenceMonth') AS date)
  LEFT JOIN
    `lemoncake-prod.full_layer.dynamodb_finance_graph_invoices_full` invoice
  ON
    charge.invoice_id = invoice.id -- já relaciona o mês correto
  LEFT JOIN
    `lemoncake-prod.full_layer.dynamodb_finance_graph_boletos_full` boleto
  ON
    JSON_VALUE(boleto.other_info_in_dynamodb, '$.invoiceId') = invoice.id
  LEFT JOIN
    `lemoncake-prod.full_layer.dynamodb_consortiums_graph_consortiums_full` consortiums
  ON
    JSON_VALUE(charge.provider, '$.id') = consortiums.id)
SELECT
  disco_consumer_unit_id AS `Número de instalação`,
  disco_client_id as `Número do cliente`,
  razaoSocial AS Cliente,
  baseReferenceMonth as `Mês de referência`,
  CASE
    WHEN status = 'paid' THEN 'Conta única paga'
    WHEN status IS NULL
  AND status_download_conta = 'Dados obtidos' THEN 'Gerando conta única'
    WHEN status = 'waitingPayment' AND invoice_delivered IS NULL THEN 'Conta única gerada - aguardando envio'
    WHEN status = 'waitingPayment'
  AND invoice_delivered IS NOT NULL THEN 'Conta única enviada - aguardando pagamento'
    WHEN status_download_conta <> 'Dados obtidos' THEN 'Aguardando download da conta da disco'
  ELSE
  '-'
END
  AS `Status geral do fluxo de cobrança`,
  CASE
    WHEN status = 'paid' THEN 'Pago'
    WHEN status = 'waitingPayment' THEN 'Aguardando Pagamento'
  ELSE
  status
END
  AS `Status da Conta única`,
  paymentDate AS `Data do pagamento`,
  DATETIME(invoice_delivered, 'America/Sao_Paulo') AS `Data de Envio da fatura`,
  due_date AS `Vencimento - conta única`,
  nossoNumero AS `Boleto - nosso número`,
  CONCAT('R$', CAST(amount AS int64)/100) AS `Valor total - conta única`,
  IFNULL(CONCAT('R$', CAST(total_paid AS int64)/100), '0') AS `Total Pago`,
  CONCAT('R$', CAST(valorLemon AS int64)/100) AS `Valor a ser pago para o consórcio`,
  codigo_consorcio_finance AS `Código do Consórcio`,
  consortiumId AS `id do Consórcio - backend`,
  CONCAT('R$', CAST(valorDisco AS int64)/100) AS `Valor a ser pago para a disco`,
  disco AS `Distribuidora`,
  IFNULL(barCode, 'Não foi possível obter o código de barras dessa fatura') AS `Código de barras - fatura disco`,
  IFNULL(Pix, 'Não foi possível obter o código pix dessa fatura') AS `Código pix - fatura disco`,
  discoDueDate AS `Vencimento da conta da disco`,
  refMonth AS `Mês de referência - disco`
FROM
  allData
ORDER BY
  baseReferenceMonth,
  disco_consumer_unit_id
'''

In [3]:
dados_raw = client.query(query=my_query).to_dataframe()

In [4]:
dados_raw

,Número de instalação,Número do cliente,Cliente,Mês de referência,Status geral do fluxo de cobrança,Status da Conta única,Data do pagamento,Data de Envio da fatura,Vencimento - conta única,Boleto - nosso número,...,Total Pago,Valor a ser pago para o consórcio,Código do Consórcio,id do Consórcio - backend,Valor a ser pago para a disco,Distribuidora,Código de barras - fatura disco,Código pix - fatura disco,Vencimento da conta da disco,Mês de referência - disco
0,3001783788,7204976375,Demilson De Assis Silva,2023-09-01,Conta única paga,Pago,2023-10-11,2023-09-29 16:04:19.759,2023-10-11,80663205288,...,R$622.45,R$385.16,C006,coopMg1,R$237.29,cemig,836500000028372901380074313673925118081141056721,00020126980014br.gov.bcb.pix0136dce71622-c425-4f3c-989d-34853baf9a720236Conta Ref SET/23 Vencime...,2023-10-17,2023-09-01
1,3001790746,7003115212,Jovelino Antunes Da Fonseca,2023-09-01,Conta única paga,Pago,2023-09-29,2023-09-29 14:49:25.297,2023-10-05,80350324725,...,R$38.85,R$5.54,C006,coopMg1,R$33.31,cemig,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,2023-10-10,2023-09-01
2,3002262154,7003247833,Francisco Carlos Martins,2023-09-01,Conta única paga,Pago,2023-10-06,2023-10-02 09:02:00.681,2023-10-06,80001880576,...,R$1336.37,R$720.47,C003,mg1,R$615.9,cemig,836500000069159001380033713857015117000226215440,00020126980014br.gov.bcb.pix0136dce71622-c425-4f3c-989d-34853baf9a720236Conta Ref SET/23 Vencime...,2023-10-11,2023-09-01
3,3004529478,7202110679,Maria Da Gloria Oliveira,2023-09-01,Conta única paga,Pago,2023-10-06,2023-09-29 16:03:51.379,2023-10-06,80668158504,...,R$485.96,R$315.2,C006,coopMg1,R$170.76,cemig,836900000016707601380033813402358119080849170800,00020126980014br.gov.bcb.pix0136dce71622-c425-4f3c-989d-34853baf9a720236Conta Ref SET/23 Vencime...,2023-10-11,2023-09-01
4,3005628518,7003163657,Jose Eustaquio Justino,2023-09-01,Conta única paga,Pago,2023-10-09,2023-09-29 16:04:17.864,2023-10-09,80884875928,...,R$513.17,R$502.44,C006,coopMg1,R$10.73,cemig,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,2023-10-14,2023-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,3014065917,7202057575,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,0,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None
352,3014067361,7203036827,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,0,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None
353,3014369601,7202202165,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,0,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None
354,3014590514,7008618239,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,0,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None


In [37]:
def treat_client(x):
    if x is None:
        result = ''
    else:
        result = x.upper()
    return result

clientes = dados_raw['Número de instalação']+'_'+ dados_raw['Cliente'].apply(treat_client)
dados_raw['clientes_in'] = clientes

In [42]:
dados_raw.sort_values(by='Número de instalação', ascending=True, inplace=True, ignore_index=True)


In [43]:
dados_raw[['Cliente','clientes_in']].head(100)

,Cliente,clientes_in
0,Rafael Geraldo De Araujo,3000100072_RAFAEL GERALDO DE ARAUJO
1,Rafael Geraldo De Araujo,3000100072_RAFAEL GERALDO DE ARAUJO
2,None,3000122171_
3,Elpidio Fernandes De Souza,3000122171_ELPIDIO FERNANDES DE SOUZA
4,None,3000138448_
...,...,...
95,None,3001949438_
96,Fernando Freire Dias,3001949438_FERNANDO FREIRE DIAS
97,None,3002098356_
98,Edmar Alvarenga,3002098356_EDMAR ALVARENGA


In [52]:
cond = dados_raw['Cliente'].apply(lambda x: x is None)
dados_raw.loc[cond,:]

,Número de instalação,Número do cliente,Cliente,Mês de referência,Status geral do fluxo de cobrança,Status da Conta única,Data do pagamento,Data de Envio da fatura,Vencimento - conta única,Boleto - nosso número,...,Valor a ser pago para o consórcio,Código do Consórcio,id do Consórcio - backend,Valor a ser pago para a disco,Distribuidora,Código de barras - fatura disco,Código pix - fatura disco,Vencimento da conta da disco,Mês de referência - disco,clientes_in
2,3000122171,7004572163,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3000122171_
4,3000138448,7009363858,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3000138448_
8,3000169488,7001604822,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3000169488_
12,3000201187,7010103309,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3000201187_
13,3000218923,7004485181,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3000218923_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,3014065917,7202057575,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3014065917_
348,3014067361,7203036827,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3014067361_
350,3014369601,7202202165,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3014369601_
352,3014590514,7008618239,None,2023-12-01,Aguardando download da conta da disco,None,None,NaT,NaT,None,...,None,None,None,None,None,Não foi possível obter o código de barras dessa fatura,Não foi possível obter o código pix dessa fatura,None,None,3014590514_


In [65]:
cond = dados_raw['Status geral do fluxo de cobrança'] != "Aguardando download da conta da disco"
cond2 = dados_raw['Data de Envio da fatura'] > '2023-11-09'
dados_contas_prontas = dados_raw.loc[cond&cond2,:].copy()

In [66]:
len(set(dados_contas_prontas['clientes_in']))

167

In [67]:
set(dados_contas_prontas['clientes_in'])

{'3000100072_RAFAEL GERALDO DE ARAUJO',
 '3000122171_ELPIDIO FERNANDES DE SOUZA',
 '3000138448_GLAUBER VIANA MENDONCA',
 '3000150913_EVALDO CARLOS DA SILVA',
 '3000169488_NILTON ROBERTO DE OLIVEIRA JUNIOR',
 '3000201187_JULIANA MARCIA DA SILVEIRA',
 '3000218923_LEVINDO LOURENCO NETO',
 '3000349984_RECANTO SABARA LTDA',
 '3000373612_CLAUDIONOR GUMERCINDO TEIXEIRA',
 '3000516875_RESTAURANTE E CERVEJARIA SAO JOSE LTDA',
 '3000574006_JOSE FARIDES DOS SANTOS',
 '3000626112_GISLEI ALMEIDA OLIVEIRA',
 '3000626114_CECILIA LUIZINHA ALMEIDA DE OLIVEIRA',
 '3000638888_GILBERTO ANTONIO DA SILVA',
 '3000696409_RENATO BARCELOS',
 '3000721197_CENTRO DE FORMACAO DE CONDUTORES MULTI ALCANCE LTDA',
 '3000725027_GILSON FERNANDES',
 '3000746923_MERCEARIA FORMIDAVEL LTDA',
 '3000880804_DECIO PEREIRA DOS SANTOS',
 '3000991672_MANOEL JUNIOR FERREIRA ESPERANCA',
 '3001015626_DAVID GOMES DA SILVA',
 '3001154564_MARIA DAS GRACAS DE SOUZA',
 '3001180856_IRVANIA DA SILVA MORAIS',
 '3001195946_LAZARO JOSE DE CASTR